In [ ]:
from importlib import reload
from datetime import timedelta, time
#from glob import glob
from pathlib import Path
import numpy as np

from obspy.core import UTCDateTime as UTC

import plotly.graph_objects as go

#import h5py

#from data_quality_control import processing
#from data_quality_control import sds_db
from data_quality_control import analysis, dqclogging, timelist #, util

#logger = sds_db.logger

# Only for display in documentation!
from IPython.core.display import display, HTML 

import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
#%matplotlib widget

# Seismic data

In [ ]:
starttime = UTC("2020-12-20")
endtime = UTC("2021-01-12")
#starttime = UTC("2020-01-01")
#endtime = UTC("2021-12-31")

In [ ]:
network = 'GR'
station = 'BFO'
location = ''
channel = 'BHZ'
datadir = Path('../../sample_output/run_processing/')
#datadir = "/home/lehr/Projects/data_quality_control/processed/"
#datadir = "../data/"
stationcode = "{}.{}.{}.{}".format(network, station, 
                                   location, channel)

winddatafile = Path("../../testing/winddata_bfo.txt")

In [ ]:
logfilename = datadir.joinpath("log/read_winddata.log")
dqclogging.configure_handlers(analysis.logger, "INFO", "DEBUG", logfilename)

In [ ]:
#reload(sds_db)
#reload(base)
reload(analysis)
lyza = analysis.Analyzer(datadir, stationcode,
                            fileunit="year")

In [ ]:
print(lyza)

In [ ]:
files = lyza.get_available_datafiles()
print(files)

In [ ]:
sdate, edate = lyza.get_available_timerange()
print(sdate, edate)

In [ ]:
DATA = lyza.get_data(starttime, endtime)

In [ ]:
DATA

In [ ]:
lyza.psds.shape

In [ ]:
fig = lyza.plot_spectrogram()

In [ ]:
DATA

# Get time list from wind data

In [ ]:
def read_interp_winddata(fname, stime, etime, delta):
    awind = read_winddata(fname)
    return interpolate_windarray(awind, stime, etime, delta)
    
def read_winddata(fname):
    winddata = []
    with open(fname, "r") as f:
        f.readline()
        for l in f.readlines():
            l = l.strip()
            date, time, speed = l.split()
            #datetime = UTC("{}T{}".format(date, time))
            datetime = "{}T{}".format(date, time)
            winddata.append([UTC(datetime), float(speed)])

    return np.array(winddata)

def interpolate_windarray(awind, stime, etime, delta):
    xp = np.asarray([t.timestamp for 
                    t in awind[:,0]])#, dtype=np.float_)
    fp = np.asarray(awind[:,1], dtype=np.float_)

    x = np.arange(stime.timestamp,
                 etime.timestamp+1,
                 delta)

    f = np.interp(x, xp, fp, )
    return x, f

## Read wind data

In [ ]:
reload(timelist)
awind = timelist.read_winddata(winddatafile)

##  Interpolate wind data

Wind speed is available about every 6 hours. The power spectral densities are available
every 1 hour. We want to filter the PSDs for times when wind speed is in a specific range.
To do so, we interpolate the wind speed to the finer grid.

In [ ]:
timedelta_wind = np.ediff1d(awind[:,0])
print("Report interval of wind speed is approx. {:.0f} h".format(np.mean(timedelta_wind) / 3600))

In [ ]:
x, f = read_interp_winddata(winddatafile, starttime, endtime, lyza.winlen_seconds)

In [ ]:
timedelta_wind = np.ediff1d(x)
print("New interval of wind speed is {:.1f} h".format(np.mean(timedelta_wind) / 3600))

## Filter wind speed

In [ ]:
# Extract times with certain wind speed
cmin = 3
cmax = 6
xsel = x[np.logical_and(f >= cmin, f <= cmax)]
tlist = [UTC(xi) for xi in xsel]
print("For {:d} samples of {} is wind speed between {:g} and {:g} m/s".format(
    len(tlist), len(x), cmin, cmax))

In [ ]:
#reload(sds_db)
#reload(base)
reload(analysis)
lyza = analysis.Analyzer(datadir, stationcode,
                            fileunit="year")

In [ ]:
DATA = lyza.get_data(tlist)
print(lyza.starttime, lyza.endtime)
print(lyza.amps.shape)
print("PSD shape:", lyza.psds.shape)
print("len(tlist):", len(tlist))

fig = lyza.plot_spectrogram()#cmap=plt.cm.gist_heat, vmin=-0.7, vmax=2.4)
#fig_cont.savefig(datadir.joinpath("spectrogram_wind_{:g}-{:g}mps.png"))

In [ ]:
fname = "../../testing/winddata_bfo.txt"
winddata = []
with open(fname, "r") as f:
    f.readline()
    for l in f.readlines():
        l = l.strip()
        date, time, speed = l.split()
        #datetime = UTC("{}T{}".format(date, time))
        datetime = "{}T{}".format(date, time)
        if UTC(datetime) < UTC("2020-01-01"): continue
        #winddata.append([datetime, float(speed)])
        winddata.append([UTC(datetime), float(speed)])

In [ ]:
awind = np.array(winddata)

## Simple

No interpolation, just extract times with
dates as is.

In [ ]:
#speed = np.asarray(awind[:,1], dtype=np.float64)
#idx = np.where(speed <= 2)[0]
#selected_wind = awind[idx, :]
selected_wind = awind[awind[:,1] >= 4.0]

In [ ]:
selected_wind.shape

In [ ]:
selected_wind

In [ ]:
reload(analysis)
lyza = analysis.Analyzer(datadir, stationcode,
                            fileunit="year")
tlist = selected_wind[:,0]
DATA = lyza.get_data(tlist)

print(lyza.starttime, lyza.endtime)
print(lyza.amps.shape)
print("PSD shape:", lyza.psds.shape)
print("len(tlist):", len(tlist))

In [ ]:
lyza.starttime, lyza.endtime

In [ ]:
iamps, ipsds = util._get_data_indices(DATA, tlist)

In [ ]:
len(ipsds) 

In [ ]:
fig = lyza.plot_spectrogram()

In [ ]:
lyza.psds.shape

## Interpolated

The wind data is only available every 6 hours 
whereas the seismic data is processed every 1 hour.
In order to not loose information, we interpolate
the windspeeds to the time grid of the seismic data
before extracting the times of the targeted wind
ranges.

In [ ]:
#awind_tstamps = awind.copy()
xp = np.asarray([t.timestamp for 
                t in awind[:,0]])#, dtype=np.float_)
fp = np.asarray(awind[:,1], dtype=np.float_)

In [ ]:
stime = starttime #UTC("2020-01-01")
etime = endtime #UTC("2021-12-31")
x = np.arange(stime.timestamp,
             etime.timestamp+1,
             lyza.winlen_seconds)
print(x.shape)

In [ ]:
x.dtype, fp.dtype, xp.dtype

In [ ]:
f = np.interp(x, xp, fp, )

In [ ]:
xsel = x[f >= 3]
tlist = [UTC(xi) for xi in xsel]

In [ ]:
DATA = lyza.get_data(tlist)

print(lyza.starttime, lyza.endtime)
print(lyza.amps.shape)
print("PSD shape:", lyza.psds.shape)
print("len(tlist):", len(tlist))

In [ ]:
fig = lyza.plot_spectrogram()

In [ ]:
tstr = "2022-12-01T00:12:00"

In [ ]:
tnp = np.datetime64(tstr)
print(tnp)
tutc = UTC(tstr)
print(tutc)

In [ ]:
str(tutc.datetime)

In [ ]:
np.datetime64(str(tutc.datetime))

In [ ]:
UTC(str(tnp))

In [ ]:
tstamp = tutc.timestamp
np.datetime64(int(tstamp), "s")

In [ ]:
t = np.datetime64(awind[0,0])

In [ ]:
tax = np.arange(np.datetime64(awind[0,0]),
                np.datetime64(awind[-1,0]), 3600, "datetime64[s]")
#tax = np.array([t])

In [ ]:
tax.shape[0]

In [ ]:
tax.shape

In [ ]:
np.float64(tax)

In [ ]:
np.float_(awind[:,0])

In [ ]:
np.float_(awind[:,0])

In [ ]:
t = tax[0]

In [ ]:
y = np.interp(np.float_(tax), np.float_(awind[:,0]), 
              np.float_(awind[:,1]), )

In [ ]:
y.shape

In [ ]:
np.asarray(awind[:,0], dtype=np.datetime64)

In [ ]:
np.asarray(awind[:,1], np.float64)

In [ ]:
plt.plot([np.datetime64(t) for t in awind[:,0]], speed)
plt.plot([np.datetime64(t) for t in selected_wind[:,0]], 
            np.asarray(selected_wind[:,1], np.float64))

In [ ]:
tlist = selected_wind[:,0]

In [ ]:
awind = np.array(winddata)
windtime = np.array([np.datetime64(t) for t, v in winddata])

#speed = np.asarray(awind[:,1], dtype=np.float64)
#idx = np.where(speed <= 2)[0]
#selected_wind = awind[idx, :]

In [ ]:
plt.plot([np.datetime64(t) for t in awind[:,0]], speed)
plt.plot([np.datetime64(t) for t in selected_wind[:,0]], 
            np.asarray(selected_wind[:,1], np.float64))

In [ ]:
tlist = selected_wind[:,0]

In [ ]:
tlist_utc = [UTC(t) for t in tlist]

In [ ]:
DATA = lyza.get_data(tlist_utc) #, stime="18:00", etime="05:00")

print(lyza.starttime, lyza.endtime)
print(lyza.amps.shape)
print(lyza.psds.shape)

In [ ]:
fig = lyza.plot_spectrogram(cmap=plt.cm.gist_heat)
plt.ylim(0.5, 8)

In [ ]:
tax = np.array(lyza.timeax_psd)

In [ ]:
t

In [ ]:
np.interp()

In [ ]:
type(tax)